In [ ]:
""" Practice with homophily beginning with Schelling's model
    The code in the first cells is from 
https://towardsdatascience.com/schellings-model-of-racial-segregation-4852fad06c13

Michael A. Goodrich
Brigham Young University

February 2022 and March 2023
"""

# Global parameters
N = 60       # Grid will be N x N
SIM_T = 0.2  # Similarity threshold (that is 1-τ)
EMPTY = 0.2  # Fraction of vacant properties
B_to_R = 1   # Ratio of blue to red people

print(f"{N}x{N} grid with {EMPTY} houses empty and {B_to_R} times more blue people than red people")
print(f"People want no more than {SIM_T} to be different than them")

60x60 grid with 0.3 houses empty and 1 times more blue people than red people
People want no more than 0.9 to be different than them


In [2]:
import numpy as np
# Define the NxNm matrix of colors
def rand_init(N, B_to_R, EMPTY):
    """ Random system initialisation.
    BLUE  =  0
    RED   =  1
    EMPTY = -1
    """
    vacant = int(N * N * EMPTY)
    population = int(N * N - vacant)
    blues = int(population * 1 / (1 + 1/B_to_R))
    reds = population - blues
    M = np.zeros(N*N, dtype=np.int8)
    M[:reds] = 1
    M[-vacant:] = -1
    np.random.shuffle(M)
    return M.reshape(N,N)


In [3]:
""" Size of neighborhood of interest """
# 8 nearest neighbors
KERNEL = np.array([[1, 1, 1],
                   [1, 0, 1],
                   [1, 1, 1]], dtype=np.int8)



In [4]:
""" Implement Schelling's mode more efficiently by using a 2D convolution
rather than as an agent-based model"""
# Convolution method
from scipy.signal import convolve2d


In [5]:

def evolve(M, boundary='wrap'):
    """
    Args:
        M (numpy.array): the matrix to be evolved
        boundary (str): Either wrap, fill, or symm
    If the similarity ratio of neighbours
    to the entire neighborhood population
    is lower than the SIM_T,
    then the individual moves to an empty house.
    """
    kws = dict(mode='same', boundary=boundary)
    b_neighs = convolve2d(M == 0, KERNEL, **kws)
    r_neighs = convolve2d(M == 1, KERNEL, **kws)
    neighs   = convolve2d(M != -1,  KERNEL, **kws)

    b_dissatified = (b_neighs / neighs < SIM_T) & (M == 0)
    r_dissatified = (r_neighs / neighs < SIM_T) & (M == 1)
    M[r_dissatified | b_dissatified] = - 1
    vacant = (M == -1).sum()

    n_b_dissatified, n_r_dissatified = b_dissatified.sum(), r_dissatified.sum()
    filling = -np.ones(vacant, dtype=np.int8)
    filling[:n_b_dissatified] = 0
    filling[n_b_dissatified:n_b_dissatified + n_r_dissatified] = 1
    np.random.shuffle(filling)
    M[M==-1] = filling
    return M

""" Notice that the boundary argument of scipy.signal.convolve2d provides 
an easy way to switch from fixed ('fill')  to periodic ('wrap') boundary 
conditions. In the following we shall stick to the latter."""

" Notice that the boundary argument of scipy.signal.convolve2d provides \nan easy way to switch from fixed ('fill')  to periodic ('wrap') boundary \nconditions. In the following we shall stick to the latter."

In [ ]:
""" Try to visualize.
    I'm doing this instead of preparing my slides. 

    Totally worth it!
"""

import matplotlib as mpl
mpl.use('tkagg')
from matplotlib import pyplot as plt
cmap = mpl.colors.ListedColormap(['white','cyan', 'magenta'])

# Global parameters
N = 60        # Grid will be N x N
TAU = 0.7     # "Agents desire a fraction 𝜏 of their neighborhood to be from the same group. 
              # Increasing 𝜏 corresponds to increasing the agent’s tolerance to outsiders”"
SIM_T = 1-TAU # Similarity threshold (that is 1-τ). 
EMPTY = 0.2   # Fraction of vacant properties
B_to_R = 1    # Ratio of blue to red people

M = rand_init(N, B_to_R, EMPTY)
plt.figure(1); plt.clf()
plt.pcolormesh(M,cmap=cmap,edgecolors='w', linewidths=1)
plt.ion()
plt.waitforbuttonpress(0.001)

for t in range(0,40):
    M = evolve(M)
    plt.pcolormesh(M,cmap=cmap,edgecolors='w', linewidths=1)
    plt.waitforbuttonpress(0.001)



: 